In [ ]:
import os
import tempfile

import pandas as pd
import matplotlib.pyplot as plt

from common import observations, GEOtopRun

In [ ]:
class GEOtopPlotRun(GEOtopRun):
    
    def postprocess(self, working_dir):
        sim = super().postprocess(working_dir)
        
        point_path = os.path.join(working_dir, 'point.txt')
        point = pd.read_csv(point_path, 
                          na_values=['-9999'],
                          parse_dates=True, 
                          infer_datetime_format=True,
                          index_col=0)
        point.index.rename('datetime', inplace=True)
        
        sim = sim.append(point)
        
        return sim

In [ ]:
%%time

model = GEOtopPlotRun('inputs/Matsch_B2/run',
                         exe='../geotop/build/geotop',
                         run_args={'check': True, 
                                   'capture_output': True})

model['PointAll'] = 1
model['PointOutputFileWriteEnd'] = "\"point\""

with tempfile.TemporaryDirectory() as tmpdir:
    sim = model.eval(tmpdir)

obs = observations('inputs/Matsch_B2/obs.csv')

In [ ]:
%%time

with tempfile.TemporaryDirectory() as tmpdir:
    sim = model.eval(tmpdir)

## Air Temperature

In [ ]:
sim.rename(columns={'Tair[C]': 'air_temperature'}, inplace=True)
print(f"Loss for air temperature is {obs.metric('air_temperature', sim)}")
obs.compare('air_temperature', sim, name='Air Temperature', unit='C')
plt.show()

## Rainfall Amount

In [ ]:
sim['rainfall_amount'] = sim['Prain_over_canopy[mm]'] + sim['Psnow_over_canopy[mm]']
print(f"Loss for rainfall amount is {obs.metric('rainfall_amount', sim)}")
obs.compare('air_temperature', sim, name='Rainfall amount', unit='mm')
plt.show()

## Relative Humidity

In [ ]:
print(f"Loss for relative humidity is {obs.metric('relative_humidity', sim)}")
obs.compare('relative_humidity', sim, name='Rainfall amount')
plt.show()
# try:
#     compare(obs['relative_humidity'], 
#             sim['relative_humidity'], 
#             name='Relative humidity')
#     plt.show()
# except:
#     pass

## Wind Speed

In [ ]:
try:
    compare(obs['wind_speed'], 
            sim['wind_speed'], 
            name='Wind speed', 
            unit='m/s')
    plt.show()
except:
    pass

## Upwelling Shortwave Flux

In [ ]:
try:
    compare(obs['surface_upwelling_shortwave_flux'], 
            sim['SW_out'], 
            name='Upwelling shortwave flux', 
            unit='W/m²')
    plt.show()
except:
    pass

## Upwelling Longwave Flux

In [ ]:
try:
    compare(obs['surface_upwelling_longwave_flux'], 
            sim['LW_out'], 
            name='Upwelling longwave flux', 
            unit='W/m²')
    plt.show()
except:
    pass

## Downwelling Shortwave Flux

In [ ]:
try:
    compare(obs['surface_downwelling_shortwave_flux'], 
            sim['SW_in'],
            name='Downwelling shortwave flux', 
            unit='W/m²')
    plt.show()
except:
    pass

## Downwelling Longwave Flux

In [ ]:
try:
    compare(obs['surface_downwelling_longwave_flux'], 
            sim['LW_in'], 
            name='Downwelling longwave flux', 
            unit='W/m²')
    plt.show()
except:
    pass

## Net Radiation

In [ ]:
try:
    compare(obs['net_radiation'], 
            sim['SW_net'] + sim['LW_net'],
            name='Net radiation', 
            unit='W/m²')
    plt.show()
except:
    pass

## Latent  Heat Flux in Air

In [ ]:
try:
    compare(obs['latent_heat_flux_in_air'], 
            sim['canopy_fraction'] * (sim['LEg_veg'] + sim['LEv'] - sim['LEg_unveg']) + sim['LEg_unveg'],
            name='Latent heat flux in air', 
            unit='W/m²')
    plt.show()
except:
    pass

## Sensible Heat Flux in Air

In [ ]:
try:
    compare(obs['sensible_heat_flux_in_air'], 
            sim['canopy_fraction'] * (sim['Hg_veg'] + sim['Hv'] - sim['Hg_unveg']) + sim['Hg_unveg'],
            name='Sensible flux in air',
            unit='W/m²')
    plt.show()
except:
    pass

## Soil Moisture Content at 20mm

In [ ]:
try:
    compare(obs['soil_moisture_content_20'], 
            sim[20],
            name='Soil moisture content @ 20mm',
            rel=True)
    plt.show()
except:
    pass

## Soil Moisture Content at 50mm

In [ ]:
try:
    compare(obs['soil_moisture_content_50'], 
            sim[50],
            name='Soil moisture content @ 50mm',
            rel=True)
    plt.show()
except:
    pass

## Soil Moisture Content at 200mm

In [ ]:
try:
    compare(obs['soil_moisture_content_200'], 
            sim[200],
            name='Soil moisture content @ 200mm',
            rel=True)
    plt.show()
except:
    pass